# Development

Agreement and policy: https://developer.twitter.com/en/developer-terms/agreement-and-policy

In [2]:
import os
os.chdir('..')

from modules import TwitterDataCleaner, get_tweets

## Get tweets

In [ ]:
query = 'sandra torres'
start_time = "2023-04-26T00:00:00Z" 
end_time = "2023-04-30T00:00:00Z" 
max_results = 10

tweets_call = get_tweets(
    query=query,
    start_time=start_time,
    end_time=end_time,
    max_results=max_results,
)

data_cleaner = TwitterDataCleaner(tweets_call)
tweets, users = data_cleaner.run()

In [7]:
tweets.head()

,tw_tweet,tw_usuario,tw_fecha,tw_texto,tw_sensitivo,tw_retweets,tw_replies,tw_likes,tw_quotes,tw_impresiones,tw_idioma
0,1652458622545264642,1520585767831195648,2023-04-29,la sandra torres está afuera de mi colonia 💀,False,0,6,2,1,950,es
1,1652457235832766465,1138919980974653441,2023-04-29,Acabo de ver un vídeo de Sandra Torres utiliza...,False,0,2,2,0,250,es
2,1652451650622169088,1542768481237716992,2023-04-29,Hoy vi a Sandra Torres y es mas hermosa en per...,False,0,0,3,0,97,es
3,1652441341064781826,966583284,2023-04-29,La candidata a la primera magistratura del paí...,False,0,0,0,0,64,es
4,1652418255796019205,1333266047500161024,2023-04-29,Dime qué quieres que vote por Sandra Torres si...,False,1,0,4,0,275,es


## Zero-shot feature extraction with GTP-3.5

In [9]:
import re
import openai
from authenticators import OpenAIAuthenticator

gpt = OpenAIAuthenticator()
openai.api_key = gpt.api_key

In [13]:
def preprocess_text(text):
    # Remove URLs
    url_pattern = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    text = url_pattern.sub('', text)

    # Remove emojis
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # Emoticons
        u"\U0001F300-\U0001F5FF"  # Symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # Transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # Flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        "]+",
        flags=re.UNICODE,
    )
    text = emoji_pattern.sub('', text)

    # Remove newline characters
    text = text.replace('\n', ' ')

    return text

In [14]:
example = tweets.tw_texto[9]

tweet = preprocess_text(example)
tweet

'El antivoto no es para Sandra Torres, el antivoto es para la izquierda, Sandra es de la Internacional Socialista, hasta fue su presidenta, de quedar otro partido de izquierda para segunda vuelta, se lleva el antivoto. '

In [ ]:
prompt = f"""
El siguiente es un tweet en español. Por favor, clasificalo de acuerdo a las siguientes categorías:

Valencia: [positivo, negativo, neutral]
Emoción: [felicidad, tristeza, enojo, miedo, sorpresa, disgusto, neutral]
Postura: [aprobación, desaprobación, esperanza, desilusión, indiferencia, condianza, desconfianza]
Tono: [agresivo, pasivo, asertivo, escéptico, irónico, humorístico, informativo, serio, inspiridor, otro]
Tema: [política, economía, deportes, entretenimiento, tecnología, ciencia, salud, educación, religión, cultura, medio ambiente, otro]

Además, evalúalo utilizando una escala continua con rango de 0 a 1 en las siguientes dimensiones:

Amabilidad: [0.0 - 1.0]
Legibilidad: [0.0 - 1.0]
Controversialidad: [0.0 - 1.0]
Informatividad: [0.0 - 1.0]
Espectro político: [0.0 - 1.0] (0 = izquierda, 1 = derecha)

Formatea tu respuesta como un diccionario de Python con las siguientes llaves:

[valencia, emocion, postura, tono, tema, amabilidad, legibilidad, controversialidad, informatividad]

Tweet: '''{tweet}'''
"""

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo", temperature=0): 
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
    )
    return response.choices[0].message["content"]

In [ ]:
test = get_completion(prompt, temperature=0)
test = test.replace('\n', ' ')

In [ ]:
import json
dict = json.loads(test)
dict

In [ ]:
pd.DataFrame(dict, index=[0])

## Store them in a RDS instance

In [ ]:
cursor.close()
connection.close()

In [ ]:
# Create a sample DataFrame
data = {
    'column1': [1, 2, 3],
    'column2': ['A', 'B', 'C']
}
df = pd.DataFrame(data)

# Upload the DataFrame to your PostgreSQL instance
# Replace "your_table_name" with the desired table name
df.to_sql('your_table_name', engine, if_exists='replace', index=False)
